# 드론 라이트 쇼 검색 트렌드 데이터 탐색

## 목표
- 네이버 데이터랩 CSV 데이터 로드 및 기본 정보 파악
- 데이터 품질 확인 및 전처리
- 기본 통계 분석 및 시각화

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 스타일 설정
sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8')

## 1. 데이터 로드 및 기본 정보

In [ ]:
# 데이터 로드
df = pd.read_csv('../data/naver_datalab_fixed.csv')

print("데이터 기본 정보:")
print(f"데이터 크기: {df.shape}")
print(f"컬럼: {list(df.columns)}")
print(f"\n데이터 타입:")
print(df.dtypes)
print(f"\n결측값:")
print(df.isnull().sum())

In [ ]:
# 처음 몇 행 확인
print("데이터 샘플:")
df.head(10)

In [ ]:
# 지역별 데이터 확인
print("지역별 데이터 개수:")
print(df['region'].value_counts())
print(f"\n고유 지역: {df['region'].unique()}")

## 2. 데이터 전처리

In [ ]:
# 날짜 컬럼을 datetime으로 변환
df['date'] = pd.to_datetime(df['date'])

# 날짜 범위 확인
print(f"데이터 기간: {df['date'].min()} ~ {df['date'].max()}")
print(f"총 기간: {(df['date'].max() - df['date'].min()).days + 1}일")

# 연도, 월, 요일 컬럼 추가
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.day_name()
df['quarter'] = df['date'].dt.quarter

print(f"\n연도별 데이터:")
print(df['year'].value_counts().sort_index())

## 3. 기본 통계 분석

In [ ]:
# 지역별 기본 통계
print("지역별 검색 비율 기본 통계:")
region_stats = df.groupby('region')['ratio'].describe()
region_stats

In [ ]:
# 지역별 최대값과 해당 날짜
print("지역별 최대 검색 비율과 날짜:")
for region in df['region'].unique():
    region_data = df[df['region'] == region]
    max_idx = region_data['ratio'].idxmax()
    max_date = region_data.loc[max_idx, 'date']
    max_ratio = region_data.loc[max_idx, 'ratio']
    print(f"{region}: {max_ratio:.5f} ({max_date.strftime('%Y-%m-%d')})")

## 4. 기본 시각화

In [ ]:
# 지역별 검색 비율 분포
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 박스플롯
sns.boxplot(data=df, x='region', y='ratio', ax=axes[0,0])
axes[0,0].set_title('지역별 검색 비율 분포 (박스플롯)')
axes[0,0].tick_params(axis='x', rotation=45)

# 히스토그램
for i, region in enumerate(df['region'].unique()):
    region_data = df[df['region'] == region]['ratio']
    axes[0,1].hist(region_data, alpha=0.7, label=region, bins=50)
axes[0,1].set_title('지역별 검색 비율 히스토그램')
axes[0,1].set_xlabel('검색 비율')
axes[0,1].set_ylabel('빈도')
axes[0,1].legend()

# 연도별 평균 검색 비율
yearly_avg = df.groupby(['year', 'region'])['ratio'].mean().reset_index()
sns.barplot(data=yearly_avg, x='year', y='ratio', hue='region', ax=axes[1,0])
axes[1,0].set_title('연도별 평균 검색 비율')

# 월별 평균 검색 비율
monthly_avg = df.groupby(['month', 'region'])['ratio'].mean().reset_index()
sns.lineplot(data=monthly_avg, x='month', y='ratio', hue='region', ax=axes[1,1], marker='o')
axes[1,1].set_title('월별 평균 검색 비율')
axes[1,1].set_xlabel('월')

plt.tight_layout()
plt.savefig('../results/figures/basic_statistics.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. 시계열 데이터 시각화

In [ ]:
# Plotly를 사용한 인터랙티브 시계열 차트
fig = px.line(df, x='date', y='ratio', color='region', 
              title='지역별 드론 라이트 쇼 검색 트렌드 (2023-2025)',
              labels={'ratio': '검색 비율', 'date': '날짜', 'region': '지역'})

fig.update_layout(
    width=1200,
    height=600,
    hovermode='x unified'
)

fig.show()

In [ ]:
# 각 지역별 개별 시계열 차트
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=df['region'].unique(),
    shared_xaxes=True,
    vertical_spacing=0.08
)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for i, region in enumerate(df['region'].unique()):
    region_data = df[df['region'] == region]
    fig.add_trace(
        go.Scatter(
            x=region_data['date'],
            y=region_data['ratio'],
            mode='lines',
            name=region,
            line=dict(color=colors[i]),
            hovertemplate='%{x}<br>검색비율: %{y:.4f}<extra></extra>'
        ),
        row=i+1, col=1
    )

fig.update_layout(
    height=900,
    title_text="지역별 드론 라이트 쇼 검색 트렌드 상세",
    showlegend=False
)

fig.update_xaxes(title_text="날짜", row=3, col=1)
fig.update_yaxes(title_text="검색 비율")

fig.show()

## 6. 데이터 저장

In [ ]:
# 전처리된 데이터 저장
df.to_csv('../data/processed_data.csv', index=False)
print("전처리된 데이터가 저장되었습니다: ../data/processed_data.csv")

# 기본 통계 정보 저장
region_stats.to_csv('../results/reports/basic_statistics.csv')
print("기본 통계 정보가 저장되었습니다: ../results/reports/basic_statistics.csv")